## ЛАБОРАТОРНАЯ РАБОТА №7.
## ТЕСТИРОВАНИЕ ГИПОТЕЗ О РАСПРЕДЛЕНИЯХ
#### по дисциплине «Математическая статистика»
Направление подготовки 01.03.02 Прикладная математика и информатика
Очной формы обучения

выполнил: студент группы Б9123-01.03.02ии

Иванов Матвей Олегович

принял: Деревягин Андрей Алексеевич

# Математическая часть

### Коэффициент корреляции Пирсона
$\widehat{p} = \frac{\sum_{i=1}^{n} {(X_i-\overline{X})*(Y_i-\overline{Y})}}{\sqrt{\sum_{i=1}^{n} {(X_i-\overline{X})^2} * \sum_{i=1}^{n} {(Y_i-\overline{Y})^2}}}$

- $|\widehat{p}| \le 1$
- показывает линейную зависимость между X и Y
- Стоит использовать в случае нормального распределения выборок


In [1]:
import numpy as np
import scipy.stats as stats

def pearson_corr(x, y):
    if len(x) != len(y):
        raise ValueError("Длины выборок должны быть равны")
    n = len(x)
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sqrt(np.sum((x - mean_x)**2) * np.sum((y - mean_y)**2))
    if denominator == 0:
        return 0.0, 1.0
    r = numerator / denominator
    t = r * np.sqrt((n - 2) / (1 - r**2))
    p = 2 * stats.t.sf(np.abs(t), n-2)
    return r, p

### Коэффициент корреляции Спирмена
Ранговый аналог Пирсона. Для проверки: $H_0: \widehat{p}_{R(x), R(y)}$

- показывает монотонную зависимость между X и Y
- Стоит использовать в случае не нормального распределения выборок

In [2]:
def spearman_corr(x, y):
    def rank_data(data):
        sorted_data = sorted((val, idx) for idx, val in enumerate(data))
        ranks = [0] * len(data)
        i, n = 0, len(sorted_data)
        while i < n:
            j = i
            while j < n and sorted_data[j][0] == sorted_data[i][0]:
                    j += 1
            avg_rank = (i + j + 1) / 2.0
            for k in range(i, j):
                    ranks[sorted_data[k][1]] = avg_rank
            i = j
        return ranks
    rank_x = rank_data(x)
    rank_y = rank_data(y)
    return pearson_corr(rank_x, rank_y)

### Задачи

In [3]:
# № 1

X = []
Y = []
# Заполнение данных
data = [
    (35, [4, 0, 6, 7, 8, 3]),
    (45, [5, 5, 2, 10, 0, 0]),
    (55, [6, 7, 0, 0, 2, 3]),
    (65, [0, 6, 5, 4, 0, 2]),
    (75, [5, 1, 2, 4, 3, 0])
]
x_values = [100, 105, 110, 115, 120, 125]
  
for y_val, counts in data:
    for x_val, count in zip(x_values, counts):
        X += ([x_val] * count)
        Y += ([y_val] * count)

# Расчет коэффициента
r_self, p_self = pearson_corr(X, Y)
r_lib, p_lib = stats.pearsonr(X, Y)
print(f"результаты от моей реализации коэффициент - {r_self:.5}, p-значение - {p_self:.5}")
print(f"результаты от библиотеки scipy коэффициент - {r_lib:.5}, p-значение - {p_lib:.5}")

if -1 <= r_lib <= -0.7:
    print("сильная отрицательная корреляция")
elif -0.7 <= r_lib <= -0.3:
    print("умеренная отрицательная корреляция")
elif -0.3 <= r_lib <= 0.3:
    print("слабая или отсутствующая корреляция")
elif 0.3 <= r_lib <= 0.7:
    print("умеренная положительная корреляция")
elif 0.7 <= r_lib <= 1:
    print("сильная положительная корреляция")
else:
    raise ValueError()

if p_lib <= 0.05:
    print("p<=0.05 => принимаем гипотезу")
else:
    print("p>0.05 => отвергаем гипотезу")



результаты от моей реализации коэффициент - -0.1621, p-значение - 0.10711
результаты от библиотеки scipy коэффициент - -0.1621, p-значение - 0.10711
слабая или отсутствующая корреляция
p>0.05 => отвергаем гипотезу


In [4]:
# № 2

X = [98, 94, 88, 80, 76, 70, 63, 61, 60, 58, 56, 51]
Y = [99, 91, 93, 74, 78, 65, 64, 66, 52, 53, 48, 62]


print("нормальность распределения X, p =", stats.shapiro(X)[1])
print("нормальность распределения Y, p =", stats.shapiro(Y)[1])
print("x,Y распределения нормальны")

r_self, p_self = spearman_corr(X, Y)
r_lib, p_lib = stats.spearmanr(X, Y)

print(f"результаты от моей реализации коэффициент - {r_self:.5}, p-значение - {p_self:.5}")
print(f"результаты от библиотеки scipy коэффициент - {r_lib:.5}, p-значение - {p_lib:.5}")

if -1 <= r_lib <= -0.7:
    print("сильная отрицательная корреляция")
elif -0.7 <= r_lib <= -0.3:
    print("умеренная отрицательная корреляция")
elif -0.3 <= r_lib <= 0.3:
    print("слабая или отсутствующая корреляция")
elif 0.3 <= r_lib <= 0.7:
    print("умеренная положительная корреляция")
elif 0.7 <= r_lib <= 1:
    print("сильная положительная корреляция")
else:
    raise ValueError()

if p_lib <= 0.05:
    print("p<=0.05 => принимаем гипотезу")
else:
    print("p>0.05 => отвергаем гипотезу")


нормальность распределения X, p = 0.2992682032342635
нормальность распределения Y, p = 0.39402483634132013
x,Y распределения нормальны
результаты от моей реализации коэффициент - 0.91608, p-значение - 2.8428e-05
результаты от библиотеки scipy коэффициент - 0.91608, p-значение - 2.8428e-05
сильная положительная корреляция
p<=0.05 => принимаем гипотезу


Все обновления и историю изменений можно посмотреть по ссылке в моём репозитории \
https://github.com/matv864/AI_work/tree/main/matstat